In [1]:
import os
import numpy as np
import re
import pandas as pd
import requests
import configparser
import functions_1 as fn
import json
from datetime import datetime

In [2]:
a = pd.read_pickle('epop_conj_list.pkl')

In [3]:
%%time
id_list = ''
for i in a['id']:
    id_list+=(str(i)+',')

id_list = id_list[:-1]

config = configparser.ConfigParser()
config.read('SpTrack-2.ini')
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

with requests.Session() as session:
    resp = session.post('https://www.space-track.org/ajaxauth/login', data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")

    resp = session.get('https://www.space-track.org/basicspacedata/query/class/satcat/NORAD_CAT_ID/' + id_list + '/predicates/NORAD_CAT_ID,SATNAME/format/json')
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for satellites")

    sat_name = []
    sat_id = []
    retData = json.loads(resp.text)
    for e in retData:
        sat_name.append(e['SATNAME'])
        sat_id.append(e['NORAD_CAT_ID'])

session.close()

CPU times: total: 15.6 ms
Wall time: 1.08 s


In [4]:
b = [idx for idx, s in enumerate(sat_name) if 'STARLINK' in s]
id_arr = np.array([int(i) for i in sat_id])[b]
name_arr = np.array(sat_name)[b]

In [5]:
sl_df = a[a['id'].isin(id_arr)]
lookup = pd.DataFrame({'name':name_arr},index=id_arr)

name_list = []
for i in sl_df['id']:
    name_list.append(lookup.loc[i].values[0])

name_list = np.array(name_list)
sl_df = sl_df.assign(name=name_list)

In [6]:
# sl_df.to_pickle('starlink_epop_2023.pkl') # list of Starlink satellites that have recorded conjunctions with e-POP